In [ ]:
# Third Experiment for DrunkerNot

from keras.preprocessing.image import save_img
from functools import partial
from types import prepare_class
from keras import preprocessing
import matplotlib as mpl
import sklearn
import sys
import matplotlib.pyplot as plt
import keras_tuner as kt
import numpy as np
import os
import PIL
import pathlib
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models, Input
from tensorflow import keras
import tensorflow as tf
from sklearn.datasets import load_files
from keras.utils import np_utils
from keras.preprocessing import image
from sklearn import metrics
import pandas as pd
import seaborn as sn
assert tf.__version__ >= "2.2"


PROJECT_ROOT_DIR = "."
data_dir = os.path.join(PROJECT_ROOT_DIR, "TrainPhotos")
data_dir2 = os.path.join(PROJECT_ROOT_DIR, "ValPhotos")

data_dir = pathlib.Path(data_dir)
data_dir2 = pathlib.Path(data_dir2)
print(data_dir)

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

# Set Batch size to be all images as we don't care about splitting into batches right now
batch_size = 16
# IMPORTANT: Depends on what pre-trained model you choose, you will need to change these dimensions accordingly
img_height = 299
img_width = 299

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])


# Training Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

# Validation Dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir2,
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = ['DrunkProcessed', 'SoberProcessed']

# # Rows and columns are set to fit one training batch (32)
# n_rows = 4
# n_cols = 2
# plt.figure(figsize=(n_cols * 3, n_rows * 3))
# for images, labels in train_ds.take(1):
#     for i in range(n_rows*n_cols):
#         plt.subplot(n_rows, n_cols, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.axis('off')
#         plt.title(class_names[labels[i]], fontsize=12)
# plt.subplots_adjust(wspace=.2, hspace=.2)
# plt.show()

# base_model = keras.applications.EfficientNetB0(weights="imagenet",
#                                                include_top=False)

base_model = keras.applications.EfficientNetB7(
    weights='imagenet', include_top=False, classes=2)

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(2, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input,
                           outputs=output)

for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(learning_rate=.2)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=['accuracy'])

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=5)

for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=['accuracy'])


history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10)


true_y2 = []
pred_y2 = []

for x, y in val_ds:
    prediction = model.predict(x)
    pred_labels = np.argmax(prediction, axis=1).tolist()
    true_y2 += y.numpy().tolist()
    pred_y2 += pred_labels

confusion_matrix2 = metrics.confusion_matrix(y_true=true_y2, y_pred=pred_y2)
precision = metrics.precision_score(y_true=true_y2, y_pred=pred_y2)
f1 = metrics.f1_score(y_true=true_y2, y_pred=pred_y2)
recall = metrics.recall_score(y_true=true_y2, y_pred=pred_y2)

print('Precision: %f' % precision)
print('Recall: %f' % recall)
print('F1 score: %f' % f1)

df_cm = pd.DataFrame(confusion_matrix2, index=[i for i in class_names],
                     columns=[i for i in class_names])
plt.figure(figsize=(10, 7))
sn.heatmap(df_cm, annot=True)
plt.show()